In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Sep  5 04:40:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
# Enter github usename and password if the repo to be cloned is own private 
# from getpass import getpass
# user = getpass('GitHub user')
# password = getpass('GitHub password')
# os.environ['GITHUB_AUTH'] = user + ':' + password

os.chdir('/content')
%ls

drive/  sample_data/


In [4]:
# clone a private repo
!git clone https://github.com/sonalrpatel/rotated-object-detection-yolo

Cloning into 'rotated-object-detection-yolo'...
remote: Enumerating objects: 581, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 581 (delta 10), reused 64 (delta 9), pack-reused 515
Receiving objects: 100% (581/581), 118.15 MiB | 25.12 MiB/s, done.
Resolving deltas: 100% (303/303), done.


In [5]:
os.chdir('/content/rotated-object-detection-yolo')
%ls

data/      display_inputs.py  images/  outputs/          test.py
datasets/  environment.yml    lib/     readme.md         train.py
detect.py  experiments/       model/   requirements.txt  xml2txt.py


In [6]:
# download our DarkNet weights
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zPSXWwbmNwUV4OHFuKoILByx_hetPBXM' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zPSXWwbmNwUV4OHFuKoILByx_hetPBXM" -O 'weights.zip' && rm -rf /tmp/cookies.txt

--2022-09-05 04:41:03--  https://docs.google.com/uc?export=download&confirm=t&id=1zPSXWwbmNwUV4OHFuKoILByx_hetPBXM
Resolving docs.google.com (docs.google.com)... 108.177.12.138, 108.177.12.113, 108.177.12.100, ...
Connecting to docs.google.com (docs.google.com)|108.177.12.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/28locvssjhug1be7s079ii9i064eu1la/1662352800000/10265277194529546548/*/1zPSXWwbmNwUV4OHFuKoILByx_hetPBXM?e=download&uuid=7dcf3f92-6a58-40b4-b191-59e306113709 [following]
--2022-09-05 04:41:03--  https://doc-04-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/28locvssjhug1be7s079ii9i064eu1la/1662352800000/10265277194529546548/*/1zPSXWwbmNwUV4OHFuKoILByx_hetPBXM?e=download&uuid=7dcf3f92-6a58-40b4-b191-59e306113709
Resolving doc-04-bk-docs.googleusercontent.com (doc-04-bk-docs.googleusercontent.com)... 173.194.215.132,

In [7]:
# unzip the downloaded weights.zip
!unzip /content/rotated-object-detection-yolo/weights.zip

Archive:  /content/rotated-object-detection-yolo/weights.zip
   creating: weights/
   creating: weights/pretrained/
  inflating: weights/pretrained/yolov4.pth  


In [ ]:
# load data from zip to data
# !mkdir data/ps
!unzip /content/drive/MyDrive/10_Python/00_Datasets/DOTA-v1.0-Val.zip -d data/DOTA/

In [ ]:
# install packages
!pip install -r requirements.txt

In [13]:
import torch
print(torch.backends.cuda.is_built())
print(torch.backends.cudnn.version())
print(torch.backends.cudnn.is_available())
print(torch.backends.cudnn.enabled)

True
8302
True
True


In [14]:
# train model
!python train.py

Streaming output truncated to the last 5000 lines.
| Step: 11/58 2447/2900 | loss               | reg_loss           | conf_loss           | cls_loss            |
+-----------------------+--------------------+--------------------+---------------------+---------------------+
| YoloLayer1            | 0.8065341114997864 | 0.5509650111198425 | 0.1717791110277176  | 0.08379002660512924 |
| YoloLayer2            | 1.0390769243240356 | 0.7475135922431946 | 0.19754621386528015 | 0.09401717782020569 |
| YoloLayer3            | 1.5103188753128052 | 1.1374255418777466 | 0.2545778751373291  | 0.11831546574831009 |
+-----------------------+--------------------+--------------------+---------------------+---------------------+
Total Loss: 3.355930, Runtime: 23088.151022


---- [Epoch 43/50] ----
+-----------------------+--------------------+--------------------+---------------------+---------------------+
| Step: 12/58 2448/2900 | loss               | reg_loss           | conf_loss           | cls_l